In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pandas as pd
import math
import sklearn.preprocessing as sk
import seaborn as sns
from sklearn import metrics
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from utils import AllTripletSelector,HardestNegativeTripletSelector, RandomNegativeTripletSelector, SemihardNegativeTripletSelector # Strategies for selecting triplets within a minibatch
from metrics import AverageNonzeroTripletsMetric
from torch.utils.data.sampler import WeightedRandomSampler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
import random
from random import randint
from sklearn.model_selection import StratifiedKFold

In [2]:
# import the GDSC and PDX data needed for training and testing
# Note that the moel will be entirely trained on GDSC data, and tested on PDX data as mentioned in the paper
# The hyper parameter combinations used were the final best performing ones as mntioned in table S3 of the supplemantary materials paper

In [3]:
# GDSC data
# expression
GDSCE = pd.read_csv("/common/statsgeneral/gayara/MOLI/Cetuximab/all_data/GDSC_exprs.Cetuximab.eb_with.PDX_exprs.Cetuximab.tsv", 
                    sep = "\t", index_col=0, decimal = ",")
GDSCE = pd.DataFrame.transpose(GDSCE)
GDSCE = GDSCE.drop_duplicates()

# response
GDSCR = pd.read_csv("/common/statsgeneral/gayara/MOLI/Cetuximab/all_data/GDSC_response.Cetuximab.tsv", 
                    sep = "\t", index_col=0, decimal = ",")
# conert the int type index here to string
GDSCR.index = GDSCR.index.map(str)

# Mutation
GDSCM = pd.read_csv("/common/statsgeneral/gayara/MOLI/Cetuximab/all_data/GDSC_mutations.Cetuximab.tsv", 
                    sep = "\t", index_col=0, decimal = ".")
GDSCM = pd.DataFrame.transpose(GDSCM)

# CNA
GDSCC = pd.read_csv("/common/statsgeneral/gayara/MOLI/Cetuximab/all_data/GDSC_CNA.Cetuximab.tsv", 
                    sep = "\t", index_col=0, decimal = ".")
GDSCC.drop_duplicates(keep='last')
GDSCC = pd.DataFrame.transpose(GDSCC)

In [4]:
# PDX data
# expression
PDXE = pd.read_csv("/common/statsgeneral/gayara/MOLI/Cetuximab/all_data/PDX_exprs.Cetuximab.eb_with.GDSC_exprs.Cetuximab.tsv", 
                   sep = "\t", index_col=0, decimal = ",")
PDXE = pd.DataFrame.transpose(PDXE)

# mutation
PDXM = pd.read_csv("/common/statsgeneral/gayara/MOLI/Cetuximab/all_data/PDX_mutations.Cetuximab.tsv", 
                   sep = "\t", index_col=0, decimal = ".")
PDXM = pd.DataFrame.transpose(PDXM)

# CNA
PDXC = pd.read_csv("/common/statsgeneral/gayara/MOLI/Cetuximab/all_data/PDX_CNA.Cetuximab.tsv", 
                   sep = "\t", index_col=0, decimal = ".")
PDXC.drop_duplicates(keep='last')
PDXC = pd.DataFrame.transpose(PDXC)

# Response
# response
PDXR = pd.read_csv("/common/statsgeneral/gayara/MOLI/Cetuximab/all_data/PDX_response.Cetuximab.tsv", 
                    sep = "\t", index_col=0, decimal = ",")
# conert the int type index here to string
PDXR.index = PDXR.index.map(str)

In [5]:
PDXR.head()

,drug,response,ResponseCategory,Treatment,Treatment target,Treatment type,BestResponse,Day_BestResponse,BestAvgResponse,Day_BestAvgResponse,TimeToDouble,Day_Last,exprs,CNA,mutations
sample_name,,,,,,,,,,,,,,,
X-1027,Cetuximab,R,PD,cetuximab,EGFR,single,499.1,21,83.125,11,7.390018484288355,24,1,1,1
X-1119,Cetuximab,R,PD,cetuximab,EGFR,single,241.8,20,60.920000000000016,13,10.361563517915311,31,1,1,1
X-1156,Cetuximab,R,PD,cetuximab,EGFR,single,674.5,21,125.8,14,6.983031674208144,24,1,1,1
X-1167,Cetuximab,R,PD,cetuximab,EGFR,single,393.6,20,96.88,14,6.698630136986304,27,1,1,1
X-1172,Cetuximab,R,PD,cetuximab,EGFR,single,293.3,20,79.375,11,5.593830334190232,20,1,1,1


In [6]:
PDXR.shape

(60, 15)

In [7]:
PDXC.head()

ENTREZID,1,2,3,9,10,12,13,14,15,16,...,101060321,101340250,101340251,101340252,102723547,102724473,103091865,105375355,109623460,109731405
X-1027,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X-1119,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
X-1156,1,1,1,-1,-1,-1,1,1,-1,-1,...,-1,-1,-1,1,0,-1,-1,0,-1,-1
X-1167,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X-1172,0,1,1,-1,-1,1,1,1,1,0,...,1,1,1,-1,-1,1,-1,1,-1,0


In [8]:
# Are the indices in the PDX data the same
(PDXR.index == PDXC.index).sum()
# They sum to 60, so they are the same

60

In [9]:
# Check with all other PDX datasets
# expression
(PDXR.index == PDXE.index).sum()

60

In [10]:
# mutation
(PDXR.index == PDXM.index).sum()

60

In [11]:
# select only the needed GDSCE features based on the variance ruel they mentioned
selector = VarianceThreshold(0.05)
selector.fit_transform(GDSCE)
GDSCE = GDSCE[GDSCE.columns[selector.get_support(indices=True)]]

In [12]:
GDSCE.shape

(861, 16244)

In [13]:
PDXM.isna().sum().sum()

0

In [14]:
PDXC = PDXC.fillna(0)
PDXC[PDXC != 0.0] = 1
PDXM = PDXM.fillna(0)
PDXM[PDXM != 0.0] = 1
GDSCM = GDSCM.fillna(0)
GDSCM[GDSCM != 0.0] = 1
GDSCC = GDSCC.fillna(0)
GDSCC[GDSCC != 0.0] = 1

In [15]:
ls = GDSCE.columns.intersection(GDSCM.columns)
ls = ls.intersection(GDSCC.columns)
ls = ls.intersection(PDXE.columns)
ls = ls.intersection(PDXM.columns)
ls = ls.intersection(PDXC.columns)
ls2 = GDSCE.index.intersection(GDSCM.index)
ls2 = ls2.intersection(GDSCC.index)
ls3 = PDXE.index.intersection(PDXM.index)
ls3 = ls3.intersection(PDXC.index)
ls = pd.unique(ls)

In [16]:
ls.shape

(13348,)

In [17]:
ls2.shape

(856,)

In [18]:
ls3.shape

(60,)

In [19]:
# Choose the final feature datasets
PDXE = PDXE.loc[ls3,ls]
PDXM = PDXM.loc[ls3,ls]
PDXC = PDXC.loc[ls3,ls]
GDSCE = GDSCE.loc[ls2,ls]
GDSCM = GDSCM.loc[ls2,ls]
GDSCC = GDSCC.loc[ls2,ls]

In [20]:
GDSCR.loc[GDSCR.iloc[:,0] == 'R'] = 0
GDSCR.loc[GDSCR.iloc[:,0] == 'S'] = 1
GDSCR.columns = ['targets', 'target', 'target', 'target', 'target', 'target']
GDSCR = GDSCR.loc[ls2,:]

In [21]:
GDSCR.head()

,targets,target,target,target,target,target
683665,0,0,0,0,0,0
684052,0,0,0,0,0,0
684055,1,1,1,1,1,1
684057,0,0,0,0,0,0
684059,0,0,0,0,0,0


In [22]:
PDXR.head()

,drug,response,ResponseCategory,Treatment,Treatment target,Treatment type,BestResponse,Day_BestResponse,BestAvgResponse,Day_BestAvgResponse,TimeToDouble,Day_Last,exprs,CNA,mutations
sample_name,,,,,,,,,,,,,,,
X-1027,Cetuximab,R,PD,cetuximab,EGFR,single,499.1,21,83.125,11,7.390018484288355,24,1,1,1
X-1119,Cetuximab,R,PD,cetuximab,EGFR,single,241.8,20,60.920000000000016,13,10.361563517915311,31,1,1,1
X-1156,Cetuximab,R,PD,cetuximab,EGFR,single,674.5,21,125.8,14,6.983031674208144,24,1,1,1
X-1167,Cetuximab,R,PD,cetuximab,EGFR,single,393.6,20,96.88,14,6.698630136986304,27,1,1,1
X-1172,Cetuximab,R,PD,cetuximab,EGFR,single,293.3,20,79.375,11,5.593830334190232,20,1,1,1


In [23]:
PDXR.shape

(60, 15)

In [24]:
# PDXR.iloc[:,1]

In [25]:
PDXR.loc[PDXR.iloc[:,1] == 'R'].shape

(55, 15)

In [26]:
# need to get the same for the PDXR data
PDXR.loc[PDXR.iloc[:,1] == 'R'] = 0
PDXR.loc[PDXR.iloc[:,1] == 'S'] = 1
PDXR.columns = ['targets', 'target', 'target', 'target', 'target', 'target', 'target', 'target', 'target', 'target', 'target', 'target', 'target', 'target', 'target']
PDXR = PDXR.loc[ls3,:]

In [27]:
PDXR.head()

,targets,target,target,target,target,target,target,target,target,target,target,target,target,target,target
X-1027,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
X-1119,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
X-1156,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
X-1167,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
X-1172,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [28]:
PDXR.shape

(60, 15)

In [29]:
Y_train = GDSCR['targets'].values
Y_test = PDXR['targets'].values

In [30]:
# convert the type of these to integers
Y_train = Y_train.astype('int64')
Y_test = Y_test.astype('int64')

In [31]:
mbs = 32
hdm = 1024
zdm = 128
lre = 1e-05
lrm = 0.0005
lrc = 0.0001
lrCL = 5e-05
epch = 10
rate = 0.4  
wd = 0.001

X_trainE = GDSCE.values
X_testE =  PDXE.values
X_trainM = GDSCM.values
X_testM = PDXM.values
X_trainC = GDSCC.values
X_testC = PDXC.values
y_trainE = Y_train
y_testE = Y_test
      
# standardize the PDX data separate
scalerGDSC = sk.StandardScaler()
scalerGDSC.fit(X_trainE)
X_trainE = scalerGDSC.transform(X_trainE)
X_testE = scalerGDSC.transform(X_testE)
# Notice that only expression data is standardized
# This is as the mutation and the CNA data used here are binary

X_trainM = np.nan_to_num(X_trainM)
X_trainC = np.nan_to_num(X_trainC)
X_testM = np.nan_to_num(X_testM)
X_testC = np.nan_to_num(X_testC)
# np.nan_to_numpy Replace NaN with zero and infinity with large finite numbers
        
TX_testE = torch.FloatTensor(X_testE)
TX_testM = torch.FloatTensor(X_testM)
TX_testC = torch.FloatTensor(X_testC)
ty_testE = torch.FloatTensor(y_testE.astype(int))
        
#Train
class_sample_count = np.array([len(np.where(y_trainE==t)[0]) for t in np.unique(y_trainE)])
weight = 1. / class_sample_count
samples_weight = np.array([weight[t] for t in y_trainE])

samples_weight = torch.from_numpy(samples_weight)
sampler = WeightedRandomSampler(samples_weight.type('torch.DoubleTensor'), len(samples_weight), replacement=True)

mb_size = mbs

trainDataset = torch.utils.data.TensorDataset(torch.FloatTensor(X_trainE), torch.FloatTensor(X_trainM), 
                                                      torch.FloatTensor(X_trainC), torch.FloatTensor(y_trainE.astype(int)))

trainLoader = torch.utils.data.DataLoader(dataset = trainDataset, batch_size=mb_size, shuffle=False, num_workers=1, sampler = sampler)

n_sampE, IE_dim = X_trainE.shape
n_sampM, IM_dim = X_trainM.shape
n_sampC, IC_dim = X_trainC.shape

h_dim = hdm
Z_dim = zdm
Z_in = h_dim + h_dim + h_dim
lrE = lre
lrM = lrm
lrC = lrc
epoch = epch

costtr = []
auctr = []
costts = []
aucts = []

class AEE(nn.Module):
    def __init__(self):
        super(AEE, self).__init__()
        self.EnE = torch.nn.Sequential(
            nn.Linear(IE_dim, h_dim),
            nn.BatchNorm1d(h_dim),
            nn.ReLU(),
            nn.Dropout())
    def forward(self, x):
        output = self.EnE(x)
        return output

class AEM(nn.Module):
    def __init__(self):
        super(AEM, self).__init__()
        self.EnM = torch.nn.Sequential(
            nn.Linear(IM_dim, h_dim),
            nn.BatchNorm1d(h_dim),
            nn.ReLU(),
            nn.Dropout())
    def forward(self, x):
        output = self.EnM(x)
        return output    


class AEC(nn.Module):
    def __init__(self):
        super(AEC, self).__init__()
        self.EnC = torch.nn.Sequential(
            nn.Linear(IM_dim, h_dim),
            nn.BatchNorm1d(h_dim),
            nn.ReLU(),
            nn.Dropout())
    def forward(self, x):
        output = self.EnC(x)
        return output      
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.FC = torch.nn.Sequential(
            nn.Linear(Z_in, Z_dim),
            nn.ReLU(),
            nn.Dropout(rate),
            nn.Linear(Z_dim, 1),
            nn.Dropout(rate),
            nn.Sigmoid())
    def forward(self, x):
        return self.FC(x)
        
torch.cuda.manual_seed_all(42)

AutoencoderE = AEE()
AutoencoderM = AEM()
AutoencoderC = AEC()

solverE = optim.Adagrad(AutoencoderE.parameters(), lr=lrE)
solverM = optim.Adagrad(AutoencoderM.parameters(), lr=lrM)
solverC = optim.Adagrad(AutoencoderC.parameters(), lr=lrC)

Clas = Classifier()
SolverClass = optim.Adagrad(Clas.parameters(), lr=lrCL, weight_decay = wd)
C_loss = torch.nn.BCELoss()

for it in range(epoch):

    epoch_cost4 = 0
    epoch_cost3 = []
    num_minibatches = int(n_sampE / mb_size) 

    for i, (dataE, dataM, dataC, target) in enumerate(trainLoader):
        flag = 0
        AutoencoderE.train()
        AutoencoderM.train()
        AutoencoderC.train()
        Clas.train()
        if torch.mean(target)!=0. and torch.mean(target)!=1.:                      

            ZEX = AutoencoderE(dataE)
            ZMX = AutoencoderM(dataM)
            ZCX = AutoencoderC(dataC)

            ZT = torch.cat((ZEX, ZMX, ZCX), 1)
            ZT = F.normalize(ZT, p=2, dim=0)

            Pred = Clas(ZT)
            loss = C_loss(Pred,target.view(-1,1))   

            y_true = target.view(-1,1)
            y_pred = Pred
            AUC = roc_auc_score(y_true.detach().numpy(),y_pred.detach().numpy()) 

            solverE.zero_grad()
            solverM.zero_grad()
            solverC.zero_grad()
            SolverClass.zero_grad()

            loss.backward()

            solverE.step()
            solverM.step()
            solverC.step()
            SolverClass.step()
                    
            epoch_cost4 = epoch_cost4 + (loss / num_minibatches)
            epoch_cost3.append(AUC)
            flag = 1

    if flag == 1:
        costtr.append(torch.mean(epoch_cost4))
        auctr.append(np.mean(epoch_cost3))
        print('Iter-{}; Total loss: {:.4}'.format(it, loss))

with torch.no_grad():

    AutoencoderE.eval()
    AutoencoderM.eval()
    AutoencoderC.eval()
    Clas.eval()

    ZET = AutoencoderE(TX_testE)
    ZMT = AutoencoderM(TX_testM)
    ZCT = AutoencoderC(TX_testC)

    ZTT = torch.cat((ZET, ZMT, ZCT), 1)
    ZTT = F.normalize(ZTT, p=2, dim=0)

    PredT = Clas(ZTT)
    lossT = C_loss(PredT,ty_testE.view(-1,1))         

    y_truet = ty_testE.view(-1,1)
    y_predt = PredT
    AUCt = roc_auc_score(y_truet.detach().numpy(),y_predt.detach().numpy())

    costts.append(lossT)
    aucts.append(AUCt)


Iter-0; Total loss: 0.661
Iter-1; Total loss: 0.6172
Iter-2; Total loss: 0.6711
Iter-3; Total loss: 0.5673
Iter-4; Total loss: 0.6004
Iter-5; Total loss: 0.5721
Iter-6; Total loss: 0.5449
Iter-7; Total loss: 0.6193
Iter-8; Total loss: 0.5293
Iter-9; Total loss: 0.5498


In [32]:
AUCt

0.6181818181818182

In [33]:
y_predt.shape

torch.Size([60, 1])